# Chapter 06: Example Application

**Declarative coding:** Focuses on describing what you want to achieve without specifying how to do it. The system handles the implementation details (e.g., SQL).

**Imperative coding:** Focuses on describing how to achieve a result through explicit instructions or steps, manipulating the program's state (e.g., C, Java).

**Differences:**
- _Declarative:_ Focuses on the what
- _Imperative:_ Focuses on the how

In [1]:
# Initiate dependencies :
import requests
from IPython.display import display, HTML
from functools import partial, reduce
import utils
import json

First Example: 

In [2]:
class Car:
    def __init__(self, name):
        self.make = name

cars = [Car('Jaguar'), Car('Volvo')]

In [3]:
# imperative
makes = []
for i in range(len(cars)):
    makes.append(cars[i].make)
makes

['Jaguar', 'Volvo']

In [4]:
# declarative
makes = list(map(lambda car: car.make, cars))
makes

['Jaguar', 'Volvo']

Second Example:

In [5]:
users = {'test': 'pass123'}
toUser = lambda form: form['username'] if form['username'] in users and users[form['username']]==form['password'] else None 
logIn = lambda x: print(f'User {x} has logged in!') if x in users else print('User or Password Incorrect')

In [6]:
# imperative
def authenticate(form):
  user = toUser(form)
  return logIn(user)

# because there is still an encoded step-by-step evaluation within in it
authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!


In [7]:
# declarative
authenticate = utils.compose(logIn, toUser)

authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!


### Let's explore Flickr API using Functional Programming

In [8]:
host = 'api.flickr.com'
path = '/services/feeds/photos_public.gne'

query = lambda t: f'?tags={t}&format=json&jsoncallback=?'
url = lambda t: f'https://{host}{path}{query(t)}'

Let’s define our own versions of getJSON and printHTML. \
Unlike the impure functions provided in the guide (written in JavaScript), \
the functions below are pure, but we’ll keep them wrapped inside Impure for reference.

In [9]:
class Impure:
    setHtml = utils.compose(display, HTML, lambda tags: f"<div>{reduce(lambda acc, x: f'{acc}{x}', tags, '')}</div>")
    getJSON = utils.compose(json.loads, lambda res: res.text[1:-1], requests.get)
    trace = utils.trace

In [10]:
app = utils.compose(Impure.getJSON, Impure.trace('response'), url)

For a better error readability, we’ll wrap our function with errorHandle.

In [11]:
utils.errorHandle(app)('cats')

response https://api.flickr.com/services/feeds/photos_public.gne?tags=cats&format=json&jsoncallback=?


{'title': 'Recent Uploads tagged cats',
 'link': 'https://www.flickr.com/photos/tags/cats/',
 'description': '',
 'modified': '2025-01-25T19:38:59Z',
 'generator': 'https://www.flickr.com',
 'items': [{'title': 'Caturday - January 25, 2025',
   'link': 'https://www.flickr.com/photos/kevingamin/54287360374/',
   'media': {'m': 'https://live.staticflickr.com/65535/54287360374_bb93483aab_m.jpg'},
   'date_taken': '2025-01-21T09:04:33-08:00',
   'description': ' <p><a href="https://www.flickr.com/people/kevingamin/">kevingamin</a> posted a photo:</p> <p><a href="https://www.flickr.com/photos/kevingamin/54287360374/" title="Caturday - January 25, 2025"><img src="https://live.staticflickr.com/65535/54287360374_bb93483aab_m.jpg" width="240" height="180" alt="Caturday - January 25, 2025" /></a></p> <p>Pepper Ann does this to me at least once a day when I’m working. She also lets me scritch her tummy, so it’s a perfect break from work.</p> ',
   'published': '2025-01-25T19:38:59Z',
   'author':

In [12]:
# Let's define our prop
prop = lambda property, object: object[property]

In [13]:
mediaUrl = utils.compose(partial(prop, 'm'), partial(prop, 'media'))
mediaUrls = utils.compose(utils.map(mediaUrl), partial(prop, 'items'))

In [14]:
render = utils.compose(Impure.setHtml, mediaUrls)
app = utils.compose(render, Impure.getJSON, url)

In [15]:
img = lambda src: f"<img src='{src}' />"

In [16]:
images = utils.compose(utils.map(img), mediaUrls)
render = utils.compose(Impure.setHtml, images)
app = utils.compose(render, Impure.getJSON, url)

In [17]:
app("cats")

#### Summary
- We tried to explore the Flickr API to display cat images.
- We were able to maintain a minimal use of side effects, as we kept them separate from the pure codebase.
- We focused on declarative specifications to describe what the application does, rather than how it achieves it.
- We produced a clean, equation-like structure in the code, enabling easier reasoning and refactoring.

<br/>

---

### A principled Refactor

One of the optimisations to be aware of is the following:
```python
compose(map(f), map(g)) == map(compose(f, g))
```

In [18]:
# Original Code
mediaUrl = utils.compose(partial(prop, 'm'), partial(prop, 'media'))
mediaUrls = utils.compose(utils.map(mediaUrl), partial(prop, 'items'))
images = utils.compose(utils.map(img), mediaUrls)

In [19]:
# We Inline mediaUrls within images
mediaUrl = utils.compose(partial(prop, 'm'), partial(prop, 'media'))
images = utils.compose(utils.map(img), mediaUrl, partial(prop, 'items'))

In [20]:
# Final code becomes :
mediaUrl = utils.compose(partial(prop, 'm'), partial(prop, 'media'))
mediaToImg = utils.compose(img, mediaUrl)
images = utils.compose(utils.map(mediaToImg), partial(prop, 'items'))

In [21]:
# Let's rerun our code
render = utils.compose(Impure.setHtml, images)
app = utils.compose(render, Impure.getJSON, url)
app("cats")

---
##### Summary
We applied our skills to a real-world app, using a mathematical framework to improve and refactor code. \
Next, we'll need to address error handling, code branching, and making the app fully pure, safer, and more expressive.